In [170]:
import requests

# Create user

In [241]:
res = requests.get(f'http://localhost:3000/user/create/MarioBros')

In [242]:
print(res)

<Response [200]>


In [243]:
res.json()

{'data': {'username': 'MarioBros'}, 'status': 'ok'}

# Create Chat

In [213]:
user_ids = ['5f09cc2530bc499da3baa024']

In [214]:
res = requests.get(f'http://localhost:3000/chat/create/playa?user_id={user_ids}')

In [215]:
print(res)

<Response [500]>


In [216]:
res.json()

{'message': 'Chat name already exist. Change name please! :)',
 'status': 'api-error'}

# Add user in a Chat

In [279]:
user_id=['5f09d2aceb2acd917d58c282']
conversation_id = ['5f09cd1b801a2a5c6105d3b9']

In [280]:
res = requests.get(f'http://localhost:3000/chat/5f09cd1b801a2a5c6105d3b9/adduser?user_id={user_id}')

In [281]:
print(res)

<Response [500]>


In [282]:
res.json()

{'message': 'Error. user_id already in this chat', 'status': 'api-error'}